<a href="https://colab.research.google.com/github/primojr/estudos_python/blob/master/py/rede_neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importar bibliotecas 

In [32]:
from sklearn import datasets 
from sklearn.neural_network import  MLPClassifier 

# # Bibliotecas Auxiliares
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(13)

df = pd.read_csv("df.csv", delimiter=";")

# # TensorFlow e tf.keras
# import tensorflow as tf
# from tensorflow import keras


# print(tf.__version__)

In [59]:
iris = datasets.load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [55]:
type(df)
#df.head()

pandas.core.frame.DataFrame

# Separar amostra e treino

In [51]:
#df = df.loc[:,df.columns[1:]]
X = df.loc[:, df.columns[2:14]]
y = df.loc[:,['classe1']]

n_aleatorio = np.arange(df.shape[0])
np.random.shuffle(n_aleatorio)

X_train = X.loc[n_aleatorio[:4000],:]
X_test  = X.loc[n_aleatorio[4000:],:]

y_train = y.loc[n_aleatorio[:4000],:]
y_test  = y.loc[n_aleatorio[4000:],:]


In [52]:
from pandas.core.common import random_state
clf = MLPClassifier(solver = 'sgd',
                    hidden_layer_sizes=(3,) ,
                    learning_rate_init=0.01, 
                    activation='logistic',
                    max_iter = 500,
                    random_state = 1
                    )

clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
y_pred

ValueError: ignored